In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import math as math
import statsmodels.api as sm
from scipy import stats
from matplotlib import pyplot as plt
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn import metrics
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression, RFECV

## Step 1: Read in holdout data and best model

In [2]:
df= pd.read_csv('kc_house_data_test_features.csv', index_col = 0)
print(df.shape)
df.head()

(4322, 20)


,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [3]:
import pickle

In [4]:
infile  = open('model.pickle', 'rb')
model = pickle.load(infile)
infile.close()

In [5]:
print(model.intercept_)
print(len(model.coef_))

-1041617.8600283918
113


In [6]:
infile=open("other_info.pickle", "rb")
other_info = pickle.load(infile)
infile.close()

In [7]:
other_info

[['bathrooms',
  'bedrooms',
  'sqftliving_log',
  'floors',
  'waterfront',
  'sqftliving15_log',
  'built_years',
  'renovated',
  'yard',
  98058,
  98115,
  98006,
  98034,
  98052,
  98031,
  98007,
  98199,
  98027,
  98119,
  98122,
  98155,
  98055,
  98118,
  98014,
  98059,
  98004,
  98053,
  98072,
  98133,
  98092,
  98074,
  98125,
  98045,
  98106,
  98022,
  98177,
  98008,
  98002,
  98040,
  98029,
  98042,
  98001,
  98065,
  98056,
  98144,
  98188,
  98109,
  98010,
  98075,
  98198,
  98030,
  98105,
  98023,
  98028,
  98112,
  98038,
  98116,
  98103,
  98178,
  98168,
  98005,
  98077,
  98146,
  98011,
  98126,
  98108,
  98136,
  98033,
  98003,
  98117,
  98107,
  98019,
  98102,
  98032,
  98148,
  98166,
  98070,
  98024,
  98039,
  'grade_1',
  'grade_3',
  'grade_4',
  'grade_5',
  'grade_6',
  'grade_7',
  'grade_8',
  'grade_9',
  'grade_10',
  'grade_11',
  'grade_12',
  'grade_13',
  'view_0',
  'view_1',
  'view_2',
  'view_3',
  'view_4',
  'month_

In [8]:
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,8,1540,730,1967,0,98034,47.7089,-122.241,2020,10918
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,8,1160,310,2005,0,98029,47.5472,-121.998,1470,1576
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,8,1030,250,1976,0,98077,47.7427,-122.071,1160,10565
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,8,2830,0,2005,0,98059,47.4863,-122.140,2830,7916


In [9]:
df.isna().mean()

id               0.0
date             0.0
bedrooms         0.0
bathrooms        0.0
sqft_living      0.0
sqft_lot         0.0
floors           0.0
waterfront       0.0
view             0.0
condition        0.0
grade            0.0
sqft_above       0.0
sqft_basement    0.0
yr_built         0.0
yr_renovated     0.0
zipcode          0.0
lat              0.0
long             0.0
sqft_living15    0.0
sqft_lot15       0.0
dtype: float64

## Step 2: Data Cleaning & Feature Engineering

In [10]:
df['bedrooms'] = np.where(df['bedrooms']>=8, 8, df['bedrooms'])

### Dummy Data for Zip Code

In [11]:
zip_dummy = pd.get_dummies(df['zipcode'])
df = pd.concat([df, zip_dummy], axis=1)
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,98146,98148,98155,98166,98168,98177,98178,98188,98198,98199
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,...,0,0,0,0,0,0,0,0,0,0


### Grade Dummy Data

In [12]:
view_dummy = pd.get_dummies(df['grade']).rename(columns=lambda x: 'grade_' + str(x))
df = pd.concat([df, view_dummy], axis=1)
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,grade_4,grade_5,grade_6,grade_7,grade_8,grade_9,grade_10,grade_11,grade_12,grade_13
0,1974300020,20140827T000000,4,2.50,2270,11500,1.0,0,0,3,...,0,0,0,0,1,0,0,0,0,0
1,1974300020,20150218T000000,4,2.50,2270,11500,1.0,0,0,3,...,0,0,0,0,1,0,0,0,0,0
2,3630020380,20141107T000000,3,2.50,1470,1779,2.0,0,0,3,...,0,0,0,0,1,0,0,0,0,0
3,1771000290,20141203T000000,3,1.75,1280,16200,1.0,0,0,3,...,0,0,0,0,1,0,0,0,0,0
4,5126310470,20150115T000000,4,2.75,2830,8126,2.0,0,0,3,...,0,0,0,0,1,0,0,0,0,0


### Month Sold Dummy Data

_Feature 1 & 2: I created a month feature because I want to see if it affects price. Off the month feature, I made a season feature but realized that month worked better as a dummy variable so I just went witht that_

In [13]:
df['date'] = pd.to_datetime(df['date'])
df['month_sold'] = df['date'].dt.month

In [14]:
conditions = [
    df.month_sold.isin([12,1,2]),
    df.month_sold.isin([3,4,5]),
    df.month_sold.isin([6,7,8]),
    df.month_sold.isin([9,10,11])
]

choices = [
    1,
    2,
    3,
    4
]

df['seasons'] = np.select(conditions, choices, default="NaN")

## Legend: 1=Winter, 2=Spring, 3=Summer, 4=Fall

In [15]:
month_dummy = pd.get_dummies(df['month_sold']).rename(columns=lambda x: 'month_' + str(x))
df = pd.concat([df, month_dummy], axis=1)
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,1974300020,2014-08-27,4,2.50,2270,11500,1.0,0,0,3,...,0,0,0,0,0,1,0,0,0,0
1,1974300020,2015-02-18,4,2.50,2270,11500,1.0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
2,3630020380,2014-11-07,3,2.50,1470,1779,2.0,0,0,3,...,0,0,0,0,0,0,0,0,1,0
3,1771000290,2014-12-03,3,1.75,1280,16200,1.0,0,0,3,...,0,0,0,0,0,0,0,0,0,1
4,5126310470,2015-01-15,4,2.75,2830,8126,2.0,0,0,3,...,0,0,0,0,0,0,0,0,0,0


### Condition Dummy Data

In [16]:
condition_dummy = pd.get_dummies(df['condition']).rename(columns=lambda x: 'condition_' + str(x))
df = pd.concat([df, condition_dummy], axis=1)
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,month_8,month_9,month_10,month_11,month_12,condition_1,condition_2,condition_3,condition_4,condition_5
0,1974300020,2014-08-27,4,2.50,2270,11500,1.0,0,0,3,...,1,0,0,0,0,0,0,1,0,0
1,1974300020,2015-02-18,4,2.50,2270,11500,1.0,0,0,3,...,0,0,0,0,0,0,0,1,0,0
2,3630020380,2014-11-07,3,2.50,1470,1779,2.0,0,0,3,...,0,0,0,1,0,0,0,1,0,0
3,1771000290,2014-12-03,3,1.75,1280,16200,1.0,0,0,3,...,0,0,0,0,1,0,0,1,0,0
4,5126310470,2015-01-15,4,2.75,2830,8126,2.0,0,0,3,...,0,0,0,0,0,0,0,1,0,0


### View Dummy Data

In [17]:
view_dummy = pd.get_dummies(df['view']).rename(columns=lambda x: 'view_' + str(x))
df = pd.concat([df, view_dummy], axis=1)
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,condition_1,condition_2,condition_3,condition_4,condition_5,view_0,view_1,view_2,view_3,view_4
0,1974300020,2014-08-27,4,2.50,2270,11500,1.0,0,0,3,...,0,0,1,0,0,1,0,0,0,0
1,1974300020,2015-02-18,4,2.50,2270,11500,1.0,0,0,3,...,0,0,1,0,0,1,0,0,0,0
2,3630020380,2014-11-07,3,2.50,1470,1779,2.0,0,0,3,...,0,0,1,0,0,1,0,0,0,0
3,1771000290,2014-12-03,3,1.75,1280,16200,1.0,0,0,3,...,0,0,1,0,0,1,0,0,0,0
4,5126310470,2015-01-15,4,2.75,2830,8126,2.0,0,0,3,...,0,0,1,0,0,1,0,0,0,0


### 4.2) There is a column that gives the date for when the house was sold, how could this be useful in your model? How might you transform the current column to a more useful feature?

I needed this for a dummy variable above so I included above. 

### 4.3) There are columns for when the house was built and when it was renovated.  How could you use these columns to create a new column?

_Feature 3: I created a built years feature to see how old the house is_

In [18]:
## How many years since the last renovation
df.loc[df['yr_built'] != 0, 'built_years'] = 2021-df['yr_built']
df.built_years

0       54
1       54
2       16
3       45
4       16
        ..
4318    12
4319     7
4320    12
4321    17
4322    13
Name: built_years, Length: 4322, dtype: int64

_Feature 4: I created a renovated years to see if the years a place has been renovated has an impact on price_

In [19]:
## How many years since house built
df.loc[df['yr_renovated'] != 0, 'renovated_years'] = 2021-df['yr_renovated']
df['renovated_years'] = df['renovated_years'].fillna(0)

_Feature 5: I created a years to renovated to see the time between the property was built vs the time it was renovated. I thought this would have an impact on price but it didn't so I didn't include the test here_

In [20]:
## Years to Renovation
df.loc[df['yr_renovated'] != 0, 'years_to_renovation'] = df['yr_renovated']-df['yr_built']
df['years_to_renovation'] = df['years_to_renovation'].fillna(0)

_Feature 6: I made a True/False renovated column because I wanted to see if just the fact a place has been renovated has an impact on the price_

In [21]:
## renovated (yes/no)
df.loc[df['yr_renovated'] != 0, 'renovated'] = 1
df.loc[df['yr_renovated'] == 0, 'renovated'] = 0

In [22]:
df.head()

,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,...,condition_5,view_0,view_1,view_2,view_3,view_4,built_years,renovated_years,years_to_renovation,renovated
0,1974300020,2014-08-27,4,2.50,2270,11500,1.0,0,0,3,...,0,1,0,0,0,0,54,0.0,0.0,0.0
1,1974300020,2015-02-18,4,2.50,2270,11500,1.0,0,0,3,...,0,1,0,0,0,0,54,0.0,0.0,0.0
2,3630020380,2014-11-07,3,2.50,1470,1779,2.0,0,0,3,...,0,1,0,0,0,0,16,0.0,0.0,0.0
3,1771000290,2014-12-03,3,1.75,1280,16200,1.0,0,0,3,...,0,1,0,0,0,0,45,0.0,0.0,0.0
4,5126310470,2015-01-15,4,2.75,2830,8126,2.0,0,0,3,...,0,1,0,0,0,0,16,0.0,0.0,0.0


_Feature 7: I created a feature for yard size to avoid multicollinearity between sqft_lot and sqft_living_

In [23]:
df['yard'] = df.sqft_lot/(df.sqft_living/df.floors)

In [24]:
df['sqftliving_log'] = np.log(df['sqft_living'])
df['sqftliving15_log'] = np.log(df['sqft_living15'])

There were no houses with grade 3 so I added a column with 0. 

In [25]:
df['grade_3'] = 0

## Step 3: Predict the holdout set

In [26]:
final_columns = list(other_info[0])

In [27]:
len(final_columns)

113

In [36]:
final_answers = model.predict(df[final_columns])

In [37]:
final = pd.DataFrame(final_answers)

In [38]:
final.to_csv('housing_preds_donna_lee.csv')